In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import time
import math
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


In [2]:
merge_df_scaled = pd.read_csv("../raw_data/merge_df_scaled.csv")

In [3]:
merge_df_scaled

,date,id,item_id,dept_id,store_id,state_id,sales,wday,month,year,...,event_name_1_missing,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_name_2_missing,wday_sin,wday_cos,month_sin,month_cos
0,2011-01-29,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,CA_1,CA,12,1,1,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.781831,0.623490,0.500000,0.866025
1,2011-01-29,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,CA_1,CA,4,1,1,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.781831,0.623490,0.500000,0.866025
2,2011-01-29,HOBBIES_1_016_CA_1_validation,HOBBIES_1_016,HOBBIES_1,CA_1,CA,5,1,1,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.781831,0.623490,0.500000,0.866025
3,2011-01-29,HOBBIES_1_019_CA_1_validation,HOBBIES_1_019,HOBBIES_1,CA_1,CA,0,1,1,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.781831,0.623490,0.500000,0.866025
4,2011-01-29,HOBBIES_1_032_CA_1_validation,HOBBIES_1_032,HOBBIES_1,CA_1,CA,9,1,1,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.781831,0.623490,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382595,2016-04-24,FOODS_1_046_CA_2_validation,FOODS_1_046,FOODS_1,CA_2,CA,8,2,4,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.974928,-0.222521,0.866025,-0.500000
382596,2016-04-24,FOODS_1_055_CA_2_validation,FOODS_1_055,FOODS_1,CA_2,CA,4,2,4,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.974928,-0.222521,0.866025,-0.500000
382597,2016-04-24,FOODS_1_067_CA_2_validation,FOODS_1_067,FOODS_1,CA_2,CA,15,2,4,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.974928,-0.222521,0.866025,-0.500000
382598,2016-04-24,FOODS_1_085_CA_2_validation,FOODS_1_085,FOODS_1,CA_2,CA,12,2,4,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.974928,-0.222521,0.866025,-0.500000


# ARIMA with MAE

In [4]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import warnings


# Auto_ARIMA with MAE

In [7]:
from pmdarima import auto_arima

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}

# Iterate over each unique product series identified by id
for id in merge_df_scaled['id'].unique():
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id]

    # Assuming 'sales' is the column you are predicting
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    X_train = data_train.drop(columns="sales")
    y_train = data_train["sales"]
    X_test = data_test.drop(columns="sales")
    y_test = data_test["sales"]

    # Results list for the current product time-series
    results = []

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)

    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and store the error metric for the current fold
    mae = mean_absolute_error(y_test, predictions)
    results.append(mae)

    # Store the average MAE for the current product time-series
    product_results[id] = np.mean(results)
    print(f'Mean Absolute Error for {id}: {product_results[id]}')

# Optionally, you could print or analyze the average MAE across all products
average_mae_across_products = np.mean(list(product_results.values()))
print(f'Average Mean Absolute Error across all products: {average_mae_across_products}')

# Handle possible convergence warnings
warnings.filterwarnings("ignore")

Analyzing product: HOBBIES_1_008_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_008_CA_1_validation: 7.08114315184505
Analyzing product: HOBBIES_1_015_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_015_CA_1_validation: 2.104460851430395
Analyzing product: HOBBIES_1_016_CA_1_validation


...

/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOUSEHOLD_1_409_CA_1_validation: 4.256873566220329
Analyzing product: HOUSEHOLD_1_418_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOUSEHOLD_1_418_CA_1_validation: 5.770722102302898
Analyzing product: HOUSEHOLD_1_447_CA_1_validation


KeyboardInterrupt: 

In [7]:
import dask
from dask.distributed import Client
from pmdarima import auto_arima
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

# Start a Dask client
client = Client()

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}

auto_arima_models = []

# Define a function to perform Auto ARIMA
def perform_auto_arima(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)
    
    auto_arima_models.append(model)

    # Predict on the test data
    predictions = model.predict(start=data_test["sales"].iloc[0],n_periods=len(data_test))

    # Calculate and store the error metric for the current fold
    mae = mean_absolute_error(data_test["sales"], predictions)
    return mae

# Iterate over each unique product series identified by id
for id in merge_df_scaled['id'].unique()[:10]:
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id]

    # Perform Auto ARIMA in parallel
    future = client.submit(perform_auto_arima, product_data)
    mae = client.gather(future)

    # Store the average MAE for the current product time-series
    product_results[id] = np.mean(mae)
    print(f'Mean Absolute Error for {id}: {product_results[id]}')

# Optionally, you could print or analyze the average MAE across all products
average_mae_across_products = np.mean(list(product_results.values()))
print(f'Average Mean Absolute Error across all products: {average_mae_across_products}')

# Handle possible convergence warnings
warnings.filterwarnings("ignore")


Analyzing product: HOBBIES_1_008_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_008_CA_1_validation: 7.08114315184505
Analyzing product: HOBBIES_1_015_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_015_CA_1_validation: 2.104460851430395
Analyzing product: HOBBIES_1_016_CA_1_validation


...

/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_169_CA_1_validation: 5.706505206114707
Analyzing product: HOBBIES_1_178_CA_1_validation


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Mean Absolute Error for HOBBIES_1_178_CA_1_validation: 3.8980834913594964
Analyzing product: HOBBIES_1_189_CA_1_validation
Mean Absolute Error for HOBBIES_1_189_CA_1_validation: 8.035458067974762
Average Mean Absolute Error across all products: 5.220450767267617


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [8]:
average_mae_across_products

5.220450767267617

In [9]:
auto_arima_models

[]